In [27]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = OllamaLLM(model="gemma2")
output_parser = StrOutputParser()

In [136]:
store = '레스티오'
date = '2024년 11월 21일 목요일'
time = '오후 9시 10분'

In [184]:
# Step 2: 카테고리 추론 프롬프트와 체인 구성
category_prompt_template = """
네 역할은 경영 전문가야.

주어진 결제 상황을 보고 어떤 유형의 매장에서 결제가 발생할 것인지 추론해.
결제 상황:
- 매장: {store}
- 날짜: {date}
- 시간: {time}

결과는 입력된 매장에 대한 유형을 단답식으로 출력해.
출력 예시는 다음과 같아.
커피전문점
음식점
편의점
학원
주유소
전자제품 판매점
"""

category_prompt = PromptTemplate(
    input_variables=['store', 'date', 'time'],
    template=category_prompt_template
)

category_chain = category_prompt | llm | output_parser
category = category_chain.invoke({'store': store, 'date': date, 'time': time})
category

'음식점 \n'

In [ ]:
# Step 2: 예상 결제액 추론 프롬프트와 체인 구성
estimation_prompt_template = """
네 역할은 금융 전문가야.

주어진 결제 상황과 카테고리에 따라 예상되는 결제 금액을 추정해.
결제 상황:
- 매장: {store}
- 날짜: {date}
- 시간: {time}
- 업종: {category}

결과는 숫자로만 출력해.
출력 예시는 다음과 같아.
1000
5000
30000
75000
"""

estimation_prompt = PromptTemplate(
    input_variables=['store', 'date', 'time', 'category'],
    template=estimation_prompt_template
)

category_chain = estimation_prompt ㄴ| llm | output_parser
estimated_amount = category_chain.invoke({'store': store, 'date': date, 'time': time, 'category': category})
estimated_amount

'20000 \n'

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader('/workspace/card_db/국민행복.md')
loader2 = TextLoader('/workspace/card_db/처음(ANNIVERSE).md')
loader3 = TextLoader('/workspace/card_db/Mr.Life.md')

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)
split_doc3 = loader3.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2), len(split_doc3)

(3, 3, 3)

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings

embedding_model = OllamaEmbeddings(model='gemma2')

# DB 생성
db = Chroma.from_documents(
    documents=split_doc1, embedding=embedding_model, collection_name='card_db'
)

In [ ]:
# 저장할 경로 지정
DB_PATH = '/workspace/card_db/chroma_db'

# 문서를 디스크에 저장합니다. 저장시 persist_directory에 저장할 경로를 지정합니다.
persist_db = Chroma.from_documents(
    split_doc1, embedding_model, persist_directory=DB_PATH, collection_name='card_db'
)

In [ ]:
# 디스크에서 문서를 로드합니다.
persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embedding_model,
    collection_name='card_db',
)

In [ ]:
# 저장된 데이터 확인
persist_db.get()

{'ids': ['682db457-ac34-432e-a4c9-6546a1dda9b4',
  '1b75e03a-064a-4221-9d8e-84a2b292c1b9',
  '1290bb9d-0fac-4cf2-b36b-b8c17e6114ae',
  '344e7fa7-94ec-4ef5-b697-eb601535ecd4',
  'd7dae7c9-a01a-404a-afcd-f9cf8c369d8d',
  '01926625-885b-404c-ba75-53bc04d1baa5'],
 'embeddings': None,
 'documents': ["### 병원, 약국, 산후조리원 5% 할인\n- 치과와 한의원 포함 병원, 약국, 산후조리원에서 할인\n#### 제한 사항\n- 월 2회 할인 적용\n- 건당 최대 5천원 할인\n- 동물병원 할인 제외\n### 온라인 쇼핑 5% 할인\n- 쿠팡, SSG닷컴, 롯데ON, 11번가, G마켓에서 할인\n#### 제한 사항\n- 월 2회 할인 적용\n- 건당 최대 5천원 할인\n- 서비스 대상 공식 사이트(앱)에 직접 접속하여 이용하는 경우에 한하여 할인 적용\n### 교육, 육아 10% 할인\n- 어린이집/유치원 중 부모부담금, 학원, 인터넷서점(YES24, 알라딘, 교보문고)\n#### 제한 사항\n- 월 2회 할인 적용\n- 건당 최대 5천원 할인\n- 학원은 방문 결제 시(오프라인 가맹점 결제)에 적용\n- 인터넷 서점은 온라인 매장만 적용\n###  배달앱 10% 할인\n- 배달의민족, 요기요 할인\n#### 제한 사항\n- 월 2회 할인 적용\n- 건당 최대 5천원 할인\n- 서비스 대상 공식 배달앱에서 '앱 내 카드 결제’에 한하여 적용\n### 디지털 구독 50% 할인\n- 넷플릭스, 유튜브프리미엄 할인\n#### 제한 사항\n- 월 1회 할인 적용\n- 건당 최대 5천원 할인",
  '- 서비스 대상 가맹점 공식 사이트에 직접 접속하여 신청한 정기결제(자동이체) 이용 건에 한하여 서비스 적용\n### 온라인 멤버쉽 3천원 할인\n